In [ ]:
#!/usr/bin/env python

from pyCBT.providers.scrappers import investing


# TODO: implement several values-for-one-option parsing
# TODO: some datetime formats are not working: JSON is a float, ISO not recognized
def parse_args(*args, **kwargs):
    import argparse, sys, pytz
    from datetime import datetime

    parser = argparse.ArgumentParser()

    parser.add_argument(
        "calendar",
        metavar="calendar-name",
        help="The name of the calendar, e.g.: gdp-375, adp-nonfarm-employment-change-1. Defaults to eia-crude-oil-inventories-75."
    )
    parser.add_argument(
        "--from-date",
        help="Starting date in dataset. Defaults to first register in table.",
        default=datetime(1000, 1, 1).isoformat()
    )
    parser.add_argument(
        "--to-date",
        help="Closing date in dataset. Defaults to now in the given timezone."
    )
    parser.add_argument(
        "--timezone",
        help="Timezone of 'from-date' & 'to-date' also alignment for the dataset.",
        default="America/New_York"
    )
    parser.add_argument(
        "--datetime-format",
        help="The datetime format of the dataset, example: RFC3339, UNIX, JSON, or any datetime format string.",
        default="%Y-%m-%d"
    )
    parser.add_argument(
        "--save-to", "-s",
        help="Stores the dataset in the given filename. Supported extensions are .csv (default if extension is missing) or .xlsx file. If .xlsx and a second argument (comma-separated) value is given, it is taken to be the name of the sheet."
    )
    # TODO: add interactive option
    # TODO: add verbose option

    # TODO: fix help string in save-to argument
    # TODO: fix help string in to-date
    args = parser.parse_args()

    if args.to_date is None:
        args.to_date = datetime.now(tz=pytz.timezone(args.timezone)).isoformat()

    return args._get_args(), dict(args._get_kwargs())

def dump_data(*args, **kwargs):
    import os, sys, re, string
    import pandas as pd
    from pyCBT.common.files import exist
    from openpyxl import load_workbook

    dataframe, = args
    if kwargs.get("save_to") is not None:
        if "," in kwargs["save_to"]: filename, sheetname = kwargs["save_to"].split(",")
        else: filename, sheetname = kwargs.get("save_to"), "sheet_001"
        if filename.endswith(".xlsx") and exist(filename):
            # TODO: if the file exist, ask the user
            book = load_workbook(filename)
            with pd.ExcelWriter(filename, engine="openpyxl") as excel_writer:
                excel_writer.book = book
                if sheetname in book.sheetnames:
                    match = re.match("(\w+)_(\d+)", sheetname)
                    if not match: sheetname += "_{0:03d}"
                    else: sheetname = string.join([match.groups()[0], "{0:03d}"], "_")
                i = 1
                while sheetname.format(i) in book.sheetnames: i += 1
                sheetname = sheetname.format(i)
                dataframe.to_excel(excel_writer, sheet_name=sheetname)
                excel_writer.save()
            book.close()
        elif filename.endswith(".xlsx"):
            with pd.ExcelWriter(filename, engine="openpyxl") as excel_writer:
                dataframe.to_excel(excel_writer, sheet_name=sheetname)
                excel_writer.save()
        else:
            # TODO: if the file exist, ask the user
            if filename.split(".")[-1] != "csv": filename += ".csv"
            dataframe.reset_index().to_csv(filename, index=False, line_terminator=os.linesep)
    else:
        dataframe.reset_index().to_csv(sys.stdout, index=False, line_terminator=os.linesep)

    return None

if __name__ == "__main__":
    args, kwargs = parse_args()
    table = investing.get_calendar(*args, **kwargs)
    dump_data(table, **kwargs)